In [9]:
import numpy as np
import os.path as osp
import sys 

sys.path.append("/home/viktorv/Projects/3DMNN/main/models/latent_space/src")

from utils.metrics import minimum_mathing_distance, \
jsd_between_point_cloud_sets, coverage

from utils.io import snc_category_to_synth_id,\
                                        load_all_point_clouds_under_folder

In [10]:
top_in_dir = '/home/viktorv/Projects/3DMNN/data/point_cloud_sampled/data/shape_net_core_uniform_samples_2048' # Top-dir of where point-clouds are stored.
class_name = input('Give me the class name (e.g. "chair"): ').lower()
syn_id = snc_category_to_synth_id()[class_name]
class_dir = osp.join(top_in_dir , syn_id)
all_pc_data = load_all_point_clouds_under_folder(class_dir, n_threads=8, file_ending='.ply', verbose=True)

Give me the class name (e.g. "chair"): chair
<generator object files_in_subdirs at 0x7f0b612e65c8>
6778 pclouds were loaded. They belong in 1 shape-classes.


In [11]:

n_ref = 100 # size of ref_pcs.
n_sam = 150 # size of sample_pcs.
all_ids = np.arange(all_pc_data.num_examples)
ref_ids = np.random.choice(all_ids, n_ref, replace=False)
sam_ids = np.random.choice(all_ids, n_sam, replace=False)
ref_pcs = all_pc_data.point_clouds[ref_ids]
sample_pcs = all_pc_data.point_clouds[sam_ids]

In [21]:
ae_loss = 'emd'  # Which distance to use for the matchings.

if ae_loss == 'emd':
    use_EMD = True
else:
    use_EMD = False  # Will use Chamfer instead.

mmd, matched_dists = minimum_mathing_distance(sample_pcs, ref_pcs, batch_size, normalize=normalize, use_EMD=use_EMD)
cov, matched_ids = coverage(sample_pcs, ref_pcs, 100, normalize=True, use_EMD=use_EMD)
jsd = jsd_between_point_cloud_sets(sample_pcs, ref_pcs, resolution=64)

# METRICS

In [24]:
print("Minimum matching distance: " + str(mmd))
print("Coverage: " + str(cov))

# Paper RESULTS
# 0.072 0.648

Minimum matching distance: 0.07376092
Coverage: 0.65


 Met  |     MMD        | Coverage
------|----------------|--------------
 Best |        0.072   |     64.8%
 Ours |        0.073   |     65%
